<a href="https://colab.research.google.com/github/xndrbkv/mifi_ml_engineer/blob/main/smoke-detection-ecology/Smoke_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision
!pip install huggingface-hub>=0.25.0
!pip install yolov5
!pip install Pillow

In [ ]:
# Импорт необходимых библиотек
import yolov5
import torch
import matplotlib.pyplot as plt
import requests
from io import BytesIO
import numpy as np
import cv2
from PIL import Image
import warnings

In [ ]:
original_load = torch.load
torch.load = lambda *args, **kwargs: original_load(*args, **kwargs, weights_only=False)

model = yolov5.load('keremberke/yolov5n-smoke')
model.conf = 0.25
model.iou = 0.45

In [ ]:
# Подавление предупреждений о deprecated функциях в PyTorch
warnings.filterwarnings('ignore', category=FutureWarning)

# Список путей к изображениям для анализа
images_list = [
    'https://raw.githubusercontent.com/xndrbkv/mifi_ml_engineer/main/smoke-detection-ecology/1.jpg',
    'https://raw.githubusercontent.com/xndrbkv/mifi_ml_engineer/main/smoke-detection-ecology/2.jpg',
    'https://raw.githubusercontent.com/xndrbkv/mifi_ml_engineer/main/smoke-detection-ecology/3.jpg',
    'https://raw.githubusercontent.com/xndrbkv/mifi_ml_engineer/main/smoke-detection-ecology/4.jpg'
]

# Основной цикл обработки изображений
for i, img_url in enumerate(images_list):
    print(f"\n Обработка изображения {i+1}/{len(images_list)}")

    response = requests.get(img_url)
    img_pil = Image.open(BytesIO(response.content)).convert("RGB")
    img_np = np.array(img_pil)

    # Исправление проблемы "readonly" массива в Colab
    if not img_np.flags['WRITEABLE']:
      img_np = img_np.copy()

    # Выполнение инференса (детекции дыма) с помощью YOLOv5 модели
    # Параметр size=640 определяет размер изображения для обработки
    results = model(img_np, size=640)

    # Отображение результатов детекции
    results.render()

    img_rgb = results.ims[0]

    plt.figure(figsize=(6, 6))
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.show()

    display(Image.fromarray(img_rgb))